# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [18]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np 


# Import API key
from config import geoapify_key

In [19]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto colombia,10.9878,-74.9547,82.49,77,6,26.55,CO,1707704902
1,1,stanley,54.8680,-1.6985,37.27,90,20,4.61,GB,1707704512
2,2,iaciara,-14.0958,-46.6317,66.92,97,100,2.93,BR,1707704902
3,3,thompson,55.7435,-97.8558,12.36,85,100,0.00,CA,1707704217
4,4,cabinda,-5.5500,12.2000,77.14,88,100,6.91,AO,1707704902


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    size=city_data_df['Humidity'], 
    hover_cols=['City', 'Humidity'], 
    color='City',  
    cmap='Category20', 
    alpha=0.6, 
    tiles='OSM')


# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,_size,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 18) & 
                           (city_data_df['Max Temp'] <= 30) &  
                           (city_data_df['Cloudiness'] == 0) & 
                           (city_data_df['Wind Speed'] <= 6)]

# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_df=ideal_weather_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_weather_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
117,117,creel,27.7500,-107.6333,27.68,34,0,5.01,MX,1707704912
170,170,badger,64.8000,-147.5333,21.45,81,0,0.00,US,1707704514
306,306,deqen,29.9618,90.7188,24.78,33,0,1.25,CN,1707704929
416,416,alamosa,37.4695,-105.8700,21.18,79,0,4.61,US,1707704939
462,462,enoch,37.7733,-113.0244,28.65,74,0,5.75,US,1707704946


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:


# Selecting required columns and creating the new DataFrame for hotels
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Adding an empty column for "Hotel Name"
hotel_df['Hotel Name'] = ""

# Displaying sample data from the new DataFrame
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
117,creel,MX,27.7500,-107.6333,34,
170,badger,US,64.8000,-147.5333,81,
306,deqen,CN,29.9618,90.7188,33,
416,alamosa,US,37.4695,-105.8700,79,
462,enoch,US,37.7733,-113.0244,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set radius for the hotel search (in meters)
radius = 10000
params = {
    "apiKey": geoapify_key,
    "limit": 1,
    "categories": "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
creel - nearest hotel: Hotel Cascada inn
badger - nearest hotel: Hotel North Pole
deqen - nearest hotel: No hotel found
alamosa - nearest hotel: 40 Winks Inn
enoch - nearest hotel: Travelodge by Wyndham Cedar City


,City,Country,Lat,Lng,Humidity,Hotel Name
117,creel,MX,27.7500,-107.6333,34,Hotel Cascada inn
170,badger,US,64.8000,-147.5333,81,Hotel North Pole
306,deqen,CN,29.9618,90.7188,33,No hotel found
416,alamosa,US,37.4695,-105.8700,79,40 Winks Inn
462,enoch,US,37.7733,-113.0244,74,Travelodge by Wyndham Cedar City


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [32]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE


hotel_df['City'] = hotel_df['City'].astype('category')


map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size=50,  
    hover_cols=['City', 'Hotel Name'], 
    color='City',  
    cmap='Category20', 
    alpha=0.6,
    tiles='OSM',  
    legend='top_right',  
    frame_width=800,
    frame_height=400
)
# Configure the map plot
# YOUR CODE HERE

map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name)